In [35]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
def pad_images(image, filter_size):
    p = int((filter_size - 1)/2)
    image = np.pad(array=image, pad_width=p, mode='constant', constant_values=0)
    return image

def create_dataset(gray, red, green, blue, f_s):
    X = []
    y = []
    p_gray = pad_images(gray, f_s)
    p_red = pad_images(red, f_s)
    p_green = pad_images(green, f_s)
    p_blue = pad_images(blue, f_s)
    
    for i in range(0, len(p_gray)-(f_s-1)):
        for j in range(0, len(p_gray)-(f_s-1)):
            X.append(list(p_gray[i:i+f_s,j:j+f_s].flatten()))
            y.append([p_red[i:i+f_s,j:j+f_s].flatten()[int(f_s*f_s/2)], 
                      p_green[i:i+f_s,j:j+f_s].flatten()[int(f_s*f_s/2)], 
                      p_blue[i:i+f_s,j:j+f_s].flatten()[int(f_s*f_s/2)]])
    
    return X, y
    
def get_images(directory, f_s):
    exts = ["jpg", "jpeg", "png"]
    print("Opening directory {}".format(directory))
    for root, dirs, files in os.walk(directory):
        if root:
            X = []
            y = []
            file_name = []
            for f in files:
                if f.split(".")[1] in exts:
                    image = cv2.imread(os.path.join(root, f))
                    image = cv2.resize(image, (32,32), interpolation = cv2.INTER_AREA)
                    # gray image
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    # red, green, blue components
                    red, green, blue = image[:,:,2], image[:,:,1], image[:,:,0]
                    m_X, m_y = create_dataset(gray, red, green, blue, f_s)
                
                    X.append(m_X)
                    y.append(m_y)
                    file_name.append(f)
    return X, y, file_name
                

In [5]:
directory = "./Images/"
filter_size = 3
X, y, files = get_images(directory, filter_size)
print("X: ", len(X[0]))
print("y: ", len(y[0]))
print("Filename: ", files)

Opening directory ./Images/
X:  1024
y:  1024
Filename:  ['scene8.jpeg', 'scene10.jpeg', 'scene9.jpeg', 'scene2.jpeg', 'scene1.jpeg', 'scene5.jpeg', 'scene3.jpeg', 'scene7.jpeg', 'scene6.jpeg']


In [28]:
data_X = []
for sublist in X:
    for item in sublist:
        data_X.append(item)
        
data_y_red = []
data_y_green = []
data_y_blue = []

for sublist in y:
    for item in sublist:
        data_y_red.append(item[0])
        data_y_green.append(item[1])
        data_y_blue.append(item[2])

In [59]:
# Model for red
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(data_X, data_y_red, test_size=0.2, random_state=42)
scaler = StandardScaler()
scaler.fit(X_train_r)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [75]:
X_train_r = scaler.transform(X_train_r)
X_test_r = scaler.transform(X_test_r)

mlp = MLPRegressor(hidden_layer_sizes=(10,),  activation='relu', 
                   solver='adam', max_iter=500, alpha=0.003)
mlp.fit(X_train_r,y_train_r)

MLPRegressor(activation='relu', alpha=0.003, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [76]:
predictions = mlp.predict(X_test_r)
predictions

array([114.23455371, 114.23455336, 114.23456082, ..., 114.23455014,
       114.2345584 , 114.23454919])

In [60]:
# plt.scatter(X_test_r, y_test_r)
len(X_test_r)

1844